## Introduction to Data Science

## The CRISP-DM approach to Data Science Tasks

_________________________

## I. Business Understanding
                    
#### The Business Understanding phase focuses on understanding the objectives and requirements of the project.  
+ Determine business objectives:

    ```Modelar o futebol brasileiro, predizendo resultados (vitória, derrota e empate) e tentando prever o campeão brasileiro.```


+ Assess situation:

    ```Muitos dados disponíveis (tanto no site da CBF quanto em sites esportivos) e grande público em casas de apostas.```
    

+ Determine data mining goals:

    ```Coletar os dados (mandante e visitante), bem como o placar e a data do jogo.```
    

+ Produce project plan:

    ```A ideia é utilizar dados de jogos entre 2013 e 2020 para realizar a modelagem. Um método para a validação e teste é utilizar as primeiras seis temporadas para o treinamento do modelo e as duas últimas para teste de eficiência. Possivelmente utilizar dados das primeiras rodadas do campeonato em questão para modelar mais precisamente os resultados, pois os times mudam muito de um ano para o outro. Em virtude dessa grande rotatividade de jogadores, jogos mais recentes terão um peso maior na avaliação.```

+ business objectives:

o projeto objetiva a criação de um modelo de predição de resultados gerais dos jogos do futebol brasileiro em vitória, derrota ou empate, e a inferência do campeão brasileiro utilizando o modelo implementado   utilizando dados disponíveis no site da CBF e em outros sites esportivos das partidas dos jogos entre 2013 e 2020 da série A que foram coletados via scraping.


+ Assess situation:

    ```Muitos dados disponíveis (tanto no site da CBF quanto em sites esportivos) e grande público em casas de apostas.```

...


+ data mining goals:

    ```Coletar os dados (mandante e visitante), bem como o placar e a data do jogo.```
    
(quais dados vamos usar exatamente, quais as fontes)

Utilizaremos as primeiras seis temporadas para o treinamento do modelo e as duas últimas para validação dele via eficiência. Possivelmente utilizar dados das primeiras rodadas do campeonato em questão para modelar mais precisamente os resultados, pois os times mudam muito de um ano para o outro. Em virtude dessa grande rotatividade de jogadores, jogos mais recentes terão um peso maior na avaliação. 

+ Plano do Projeto:
    
    * Coletar dados do futebol brasileiro via scraping
    * Explorar e verificar a qualidade dos dados coletados
    * Selecionar, limpar e reformatar os dados que alimentarão o modelo
    * Construir e treinar o modelo de previsão de partidas
    * Validar e analisar as previsões do modelo
    * Inferir o campeão brasileiro com o modelo implementado
    * Revisar e fazer o deploy do projeto
    

## II. Data Understanding  

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

## III. Data Preparation

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 